# Du code pour vous aider à faire une évaluation automatique

Si cela peut vous être utile, du code pour estimer les valeurs de F1 et de correspondance exacte (*exact match*) entre une réponse et son texte de référence. Les fonctions sont empruntées du code accompagnant le jeu de données SQuAD. 

À noter que cette évaluation repose sur la correspondance entre des sous-chaînes de caractères entre 2 textes. 


## 1. Les fonctions de base pour comparer 2 passages de textes

In [23]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    """Normalise les 2 textes, trouve ce qu'il y a en comment et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if len(ground_truth_tokens) == 0 or len(prediction_tokens) == 0:
        return int(ground_truth_tokens == prediction_tokens)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth): 
    """Vérifie si les 2 textes sont quasi-identiques."""
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    """La fonction princiaple. Important de noter que ground_truths est une liste 
       parce qu'il peut y avoir plusieurs réponses possibles."""
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

## 2. Quelques exemples pour illustrer l'évaluation avec des passages de texte: 

In [24]:
def evaluate_demo(prediction, ground_truths): 
    """Fonction utilitaire pour illuster l'utilisation de metric_max_over_ground_truths.
       Vous pouvez créer votre propre fonction selon vos besoins. """
    exact_match = metric_max_over_ground_truths(exact_match_score, prediction, ground_truths)
    f1_value = metric_max_over_ground_truths(f1_score, prediction, ground_truths)
    print('Exact match:', exact_match, '\nF1:', f1_value)

In [25]:
prediction = "his left foot was struck by and run over by a loader"
ground_truths = ["his left foot was struck by and run over by a loader"]
evaluate_demo(prediction, ground_truths)

Exact match: True 
F1: 1.0


In [26]:
prediction = "his left foot was struck by and run over by a loader"
ground_truths = ["his right foot was not struck by anything", "his left foot was struck by and run over by a loader"]
evaluate_demo(prediction, ground_truths)

Exact match: True 
F1: 1.0


In [27]:
prediction = "his left foot was struck by"
ground_truths = ["his left foot was struck by and run over by a loader"]
evaluate_demo(prediction, ground_truths)

Exact match: False 
F1: 0.7058823529411764


In [28]:
prediction = ""
ground_truths = ["his left foot was struck by and run over by a loader"]
evaluate_demo(prediction, ground_truths)

Exact match: False 
F1: 0


In [29]:
prediction = ""
ground_truths = [""]
evaluate_demo(prediction, ground_truths)

Exact match: True 
F1: 1
